In [1]:
from Functions import normalize_data, time_series_CV_split, train_and_predict, fine_tune_alpha
import talib
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, make_scorer
from sklearn.preprocessing import MinMaxScaler 
from time import time
from math import sqrt
from sklearn.linear_model import Lasso

from matplotlib import pyplot
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)
%matplotlib inline

train_data = pd.read_csv('bitcoin_train.csv')
print('n_features:', len(train_data.iloc[0]))
print('n_samples:', len(train_data))
train_data.head()



n_features: 41
n_samples: 2735


,date,open,high,low,close,volatility,volume,marketcap,google_trends,gold,...,TXN_per_block,est_TXN_vol,cost_per_TXN,total_TXN_fees,usd_trade_vol,hash_rate,avg_block_size,difficulty,num_unique_addr,miners_revenue
0,4/1/11,0.2950,0.2999,0.2890,0.2989,0.037022,548.33,1514954.793,0.019074,1388.5,...,7.0,8950.0,1.714398,0.023855,789.544097,0.115112,0.002296,16307.42094,943,2129.2829
1,5/1/11,0.2989,0.2990,0.2900,0.2990,0.030563,106.19,1512645.832,0.019074,1368.0,...,8.0,4746.0,1.881279,0.017457,57.813954,0.121597,0.002129,16307.42094,1064,2242.4850
2,6/1/11,0.2990,0.2990,0.2900,0.2980,0.030563,1031.28,1514853.600,0.019074,1368.5,...,7.0,5052.0,2.585471,0.026185,890.419309,0.119165,0.001730,16307.42094,950,2197.6500
3,7/1/11,0.2980,0.3220,0.2975,0.3200,0.079137,13152.15,1633699.200,0.019074,1367.0,...,7.0,5304.0,2.634545,0.023675,13047.281580,0.116733,0.001584,16307.42094,887,2318.4000
4,8/1/11,0.3200,0.3229,0.3090,0.3229,0.044001,512.04,1641355.114,0.019074,1367.0,...,6.0,9216.0,2.226883,0.006458,1273.388302,0.155644,0.001875,16307.42094,1267,3099.8208


In [5]:
# Test Set
test_data = pd.read_csv('bitcoin_test.csv')
print('n_features:', len(test_data.iloc[0]))
print('n_samples:', len(test_data))
test_data.head()

n_features: 41
n_samples: 92


,date,open,high,low,close,volatility,volume,marketcap,google_trends,gold,...,TXN_per_block,est_TXN_vol,cost_per_TXN,total_TXN_fees,usd_trade_vol,hash_rate,avg_block_size,difficulty,num_unique_addr,miners_revenue
0,1/7/18,6411.68,6432.85,6289.29,6385.82,0.022569,4788259840,1.097970e+11,62,1250.45,...,1284.148936,493208885.1,58.735614,140545.4062,489427084.1,35588942.30,0.987340,5.077500e+12,368307,10494418.50
1,2/7/18,6380.38,6683.86,6305.70,6614.18,0.058242,4396930048,1.092730e+11,72,1247.80,...,1062.904762,991949405.1,75.393733,106682.7087,224327884.6,37103365.37,0.705291,5.077500e+12,341861,11673361.88
2,3/7/18,6596.66,6671.37,6447.75,6529.59,0.034094,4672309760,1.129890e+11,69,1251.75,...,1413.432624,720058629.1,57.309569,150160.0866,281178824.1,37594816.26,0.987550,5.172890e+12,433257,11271293.25
3,4/7/18,6550.87,6771.92,6450.46,6597.55,0.048633,4176689920,1.122140e+11,65,1255.65,...,1868.508621,872392592.0,45.224241,185064.5374,356469308.6,30929068.69,1.113817,5.363680e+12,456149,9617154.00
4,5/7/18,6599.71,6749.54,6546.65,6639.14,0.030521,4999240192,1.130620e+11,69,1255.50,...,1543.194030,665699858.5,54.318173,142640.2951,345023639.3,35728406.94,0.935353,5.363680e+12,435401,11089706.00


In [6]:
# Combined train and test sets
combined_data = pd.concat([train_data, test_data], ignore_index=True) # use data from 03/2018 onwards only - best stationarity
#combined_data['Price'] = combined_data['close'].shift(-1) # Dependent variable Y
print('n_features:', len(combined_data.iloc[0]))
print('n_samples:', len(combined_data))
#combined_data = combined_data.iloc[:-1,1:]
combined_data.tail()

n_features: 41
n_samples: 2827


,date,open,high,low,close,volatility,volume,marketcap,google_trends,gold,...,TXN_per_block,est_TXN_vol,cost_per_TXN,total_TXN_fees,usd_trade_vol,hash_rate,avg_block_size,difficulty,num_unique_addr,miners_revenue
2822,26/9/18,6452.79,6585.91,6397.89,6495.00,0.028964,4.437300e+09,1.115590e+11,59.0,1194.25,...,1491.792899,842041019.0,54.247914,130284.3136,390313648.8,60089527.19,0.768241,7.152630e+12,490588,13676604.30
2823,27/9/18,6495.29,6712.10,6464.95,6676.75,0.037517,4.606810e+09,1.123050e+11,58.0,1185.40,...,1754.870504,748444892.4,46.610199,130238.5658,312748554.4,49422747.22,0.898005,7.152630e+12,474079,11369486.09
2824,28/9/18,6678.75,6785.03,6598.32,6644.13,0.027904,5.014430e+09,1.154900e+11,58.0,1187.25,...,1739.062937,824319235.8,47.597177,154586.9390,283381555.4,50844984.55,0.877031,7.152630e+12,481907,11836751.48
2825,29/9/18,6643.10,6643.10,6511.65,6601.96,0.019986,4.363690e+09,1.148860e+11,48.0,1187.25,...,1650.902597,913410388.9,51.317781,193097.0923,457654461.3,54756137.20,0.855169,7.152630e+12,527057,13046981.40
2826,30/9/18,6604.71,6643.78,6566.54,6625.56,0.011694,4.002280e+09,1.142340e+11,51.0,1187.25,...,1436.440789,464342432.7,57.575879,125158.9836,373219094.7,54045018.54,0.693384,7.152630e+12,429818,12571059.90


In [16]:
data = combined_data
data['WilliamR14'] = talib.WILLR(data['high'].values, data['low'].values, data['close'].values, timeperiod=14)
data['MACD'], data['MACD_signal'], data['MACD_hist'] = talib.MACD(data['close'].values, fastperiod=12, slowperiod=26, signalperiod=9)
data['EMA30']=talib. EMA(data['close'], timeperiod=14)
data['Price'] = data['close'].shift(-1)
data = data.iloc[1823:-1,41:]
#test = data.iloc[-91:-1,41:]
#test
#len(test)
data.iloc[-93]
data.iloc[-90]


WilliamR14      -20.913773
MACD           -252.512483
MACD_signal    -328.444505
MACD_hist        75.932022
EMA30          6396.819437
Price          6529.590000
Name: 2736, dtype: float64

In [5]:
#TA lib indicators: WILLR ‘adx’, ’aroonosc’, ’cmo’, ’sar’, ’cci’, ’mfi’, ’obv’, ’stoch’, ’rsi’, ’macd’, ’bollingerBands’, ’atr’ ,‘sma’, ’ema’, ’wma’, ’dema’, ’tema’, ’trima’, ’kama’, and ’t3'


In [6]:

data['ADX'] = talib.ADX(data['high'].values, data['low'].values, data['close'].values, timeperiod=14)
data['AROONOSC'] = talib.AROONOSC(data['high'].values, data['low'].values, timeperiod=14)
data['CMO'] = talib.CMO(data['close'].values, timeperiod=14)
data['CCI'] = talib.CCI(data['high'].values, data['low'].values, data['close'].values, timeperiod=14)
data['MFI'] = talib.MFI(data['high'].values, data['low'].values, data['close'].values, data['volume'].values, timeperiod=2)
data['OBV'] = talib.OBV(data['close'].values, data['volume'].values)
data['ATR'] = talib.ATR(data['high'].values, data['low'].values, data['close'].values, timeperiod=14)
data['DEMA'] = talib.DEMA(data['close'].values, timeperiod=30)

data['TEMA'] = talib.TEMA(data['close'].values, timeperiod=30)
data['TRIMA'] = talib.TRIMA(data['close'].values, timeperiod=30)
data['KAMA'] = talib.KAMA(data['close'].values, timeperiod=30)
data['T3'] = talib.T3(data['close'].values, timeperiod=5, vfactor=0)

data['WilliamR14'] = talib.WILLR(data['high'].values, data['low'].values, data['close'].values, timeperiod=14)

data['STOCH_slowk'],  data['STOCH_slowd'] = talib.STOCH(data['high'].values, data['low'].values, data['close'].values, fastk_period=5, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)
data['RSI'] = talib.RSI(data['close'].values, timeperiod=14)
data['MACD'], data['MACD_signal'], data['MACD_hist'] = talib.MACD(data['close'].values, fastperiod=12, slowperiod=26, signalperiod=9)

data['SAR'] = talib.SAR(data['high'].values, data['low'].values, maximum=0)
data['BBANDS_upperband'], data['BBANDS_middleband'], data['BBANDS_lowerband'] = talib.BBANDS(data['close'].values, timeperiod=5, nbdevup=2, nbdevdn=2, matype=0)

data['EMA30']=talib. EMA(data['close'], timeperiod=30)

data['SMA'] = talib.SMA(data['close'].values, timeperiod=30)
data['WMA'] = talib.WMA(data['close'].values, timeperiod=30)

data['Price'] = data['close'].shift(-1)

In [295]:

data['ADX'] = talib.ADX(data['high'].values, data['low'].values, data['close'].values, timeperiod=14)
#data['AROONOSC'] = talib.AROONOSC(data['high'].values, data['low'].values, timeperiod=14)
data['CMO'] = talib.CMO(data['close'].values, timeperiod=14)
#data['CCI'] = talib.CCI(data['high'].values, data['low'].values, data['close'].values, timeperiod=14)
#data['MFI'] = talib.MFI(data['high'].values, data['low'].values, data['close'].values, data['volume'].values, timeperiod=2)
#data['OBV'] = talib.OBV(data['close'].values, data['volume'].values)
#data['ATR'] = talib.ATR(data['high'].values, data['low'].values, data['close'].values, timeperiod=14)
#data['DEMA'] = talib.DEMA(data['close'].values, timeperiod=30)

#data['TEMA'] = talib.TEMA(data['close'].values, timeperiod=30)
#data['TRIMA'] = talib.TRIMA(data['close'].values, timeperiod=30)
#data['KAMA'] = talib.KAMA(data['close'].values, timeperiod=30)
data['T3'] = talib.T3(data['close'].values, timeperiod=5, vfactor=0)

data['WilliamR14'] = talib.WILLR(data['high'].values, data['low'].values, data['close'].values, timeperiod=14)

#data['STOCH_slowk'],  data['STOCH_slowd'] = talib.STOCH(data['high'].values, data['low'].values, data['close'].values, fastk_period=5, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)
data['RSI'] = talib.RSI(data['close'].values, timeperiod=14)
data['MACD'], MACD_signal, MACD_hist = talib.MACD(data['close'].values, fastperiod=12, slowperiod=26, signalperiod=9)
#data['MACD'], data['MACD_signal'], data['MACD_hist'] = talib.MACD(data['close'].values, fastperiod=12, slowperiod=26, signalperiod=9)

#data['SAR'] = talib.SAR(data['high'].values, data['low'].values, maximum=0)
#data['BBANDS_upperband'], data['BBANDS_middleband'], data['BBANDS_lowerband'] = talib.BBANDS(data['close'].values, timeperiod=5, nbdevup=2, nbdevdn=2, matype=0)

#data['EMA30']=talib. EMA(data['close'], timeperiod=30)
#data['EMA26']=talib. EMA(data['close'], timeperiod=26)

data['SMA'] = talib.SMA(data['close'].values, timeperiod=30)
#data['WMA'] = talib.WMA(data['close'].values, timeperiod=30)

data['Price'] = data['close'].shift(-1)

In [296]:
corr = data.iloc[1823:,41:].corr()
data.iloc[1823:,41:].corr()#[(corr <= 0.1) | (corr >= -0.1)]

,ADX,CMO,T3,WilliamR14,RSI,MACD,SMA,Price
ADX,1.000000,0.371461,0.132759,0.126231,0.371461,0.230154,0.097815,0.156287
CMO,0.371461,1.000000,-0.199308,0.826827,1.000000,0.479823,-0.271152,-0.109547
T3,0.132759,-0.199308,1.000000,-0.194710,-0.199308,0.184964,0.987162,0.982799
WilliamR14,0.126231,0.826827,-0.194710,1.000000,0.826827,0.324248,-0.240947,-0.103140
RSI,0.371461,1.000000,-0.199308,0.826827,1.000000,0.479823,-0.271152,-0.109547
MACD,0.230154,0.479823,0.184964,0.324248,0.479823,1.000000,0.033610,0.278801
SMA,0.097815,-0.271152,0.987162,-0.240947,-0.271152,0.033610,1.000000,0.958048
Price,0.156287,-0.109547,0.982799,-0.103140,-0.109547,0.278801,0.958048,1.000000


In [13]:
data.iloc[-20:,41:]

,ADX,AROONOSC,CMO,CCI,MFI,OBV,ATR,DEMA,TEMA,TRIMA,...,MACD_signal,MACD_hist,SAR,BBANDS_upperband,BBANDS_middleband,BBANDS_lowerband,EMA30,SMA,WMA,Price
2715,35.128139,-57.142857,-34.835113,-234.975230,-2.672327e-13,1.574053e+11,294.845525,7297.978087,7120.168428,7667.846917,...,-257.700892,-22.097747,0.289,8065.080471,7305.616,6546.151529,7747.905863,7766.018000,7544.082946,6582.36
2716,36.992805,-64.285714,-42.971420,-198.620265,-3.073705e-13,1.527509e+11,299.919417,7181.464059,6976.474115,7625.993792,...,-271.059940,-53.436191,0.289,7918.325980,7086.440,6254.554020,7672.709355,7694.632000,7467.717914,6349.90
2717,39.053521,-71.428571,-47.981130,-175.890785,-3.074448e-13,1.476986e+11,303.213030,7047.978196,6811.176107,7586.079250,...,-291.720230,-82.641160,0.289,7627.646081,6831.436,6035.225919,7587.366816,7615.735667,7380.961011,6675.35
2718,40.536814,-71.428571,-30.673829,-114.162652,5.100046e+01,1.528373e+11,308.174956,6968.893988,6728.431132,7548.302333,...,-310.016497,-73.185066,0.289,7038.819460,6660.110,6281.400540,7528.527022,7554.568000,7320.290968,6456.58
2719,41.914157,-71.428571,-36.085261,-99.149004,5.668782e+01,1.488819e+11,303.820316,6871.145399,6619.450120,7510.801542,...,-328.655676,-74.556717,0.289,6976.991966,6594.222,6211.452034,7459.369149,7490.826333,7249.453032,6550.16
2720,43.242821,-71.428571,-31.361761,-85.217919,-4.189970e-13,1.520761e+11,295.704579,6795.562438,6544.273149,7472.814792,...,-344.290603,-62.539707,0.289,6745.305500,6522.870,6300.434500,7400.710495,7439.354333,7188.764882,6499.27
2721,44.476581,-71.428571,-32.729217,-73.118039,4.933888e+01,1.489720e+11,280.999967,6722.041439,6472.592012,7432.364250,...,-357.250126,-51.838092,0.289,6720.698357,6506.252,6291.805643,7342.553043,7380.964333,7128.114280,6734.82
2722,44.374355,-64.285714,-20.738922,-49.117052,1.000000e+02,1.530112e+11,284.818540,6686.186865,6455.968571,7390.339083,...,-363.281090,-24.123857,0.289,6794.231263,6583.236,6372.240737,7303.344460,7330.552333,7086.427548,6769.94
2723,44.027842,-42.857143,-19.012637,-26.420868,1.000000e+02,1.570683e+11,272.515788,6659.397308,6451.163593,7347.407250,...,-363.264904,0.064742,0.289,6855.503865,6602.154,6348.804135,7268.931269,7272.442000,7050.259011,6776.55
2724,43.961549,-42.857143,-18.668736,-25.026961,5.117358e+01,1.609569e+11,268.027517,6637.238466,6451.427026,7305.065292,...,-358.525918,18.955945,0.289,6901.058549,6666.148,6431.237451,7237.164735,7217.694000,7018.265978,6729.74


In [17]:
#selected = data.iloc[1823:-1,41:]
#selected
data.to_csv('DeepLearning.csv', index=False)


In [ ]:
def get_feature_ranking(feature_scores):
    indices = np.nonzero(feature_scores)[0].tolist()
    scores = np.abs(feature_scores[indices]).tolist() 
    sorted_scores, sorted_features = zip(*sorted(zip(scores, indices),reverse=True))
    return list(sorted_features)

In [302]:
%%time
# Fine tune alpha
best_alpha = fine_tune_alpha(X,Y,'Lasso')
lasso = Lasso(alpha=best_alpha)
lasso.fit(normalize_data(X,Y),Y.reshape(-1,))
lasso_features = get_feature_ranking(lasso.coef_)
print('Lasso Features Ranking:',lasso_features)
print(best_alpha)
print(best_rmse)



Lasso Features Ranking: [2, 5, 3, 6]
8.29
372.3280442209168
CPU times: user 3min 14s, sys: 1.38 s, total: 3min 15s
Wall time: 3min 20s


In [167]:
%%time
# Fine tune alpha
best_alpha = fine_tune_alpha(X,Y,'Lasso')
lasso = Lasso(alpha=best_alpha)
lasso.fit(normalize_data(X,Y),Y.reshape(-1,))
lasso_features = get_feature_ranking(lasso.coef_)
print('Lasso Features Ranking:',lasso_features)
print(best_alpha)
print(best_rmse)



Lasso Features Ranking: [13, 12, 11, 0, 5, 3]
8.29
232.65896290815454
CPU times: user 1min 13s, sys: 251 ms, total: 1min 13s
Wall time: 1min 14s
